# Latihan Membuat Machine Learning Pipeline

In [5]:
import tensorflow as tf
from tfx.components import CsvExampleGen, StatisticsGen, SchemaGen, ExampleValidator, Transform, Trainer, Tuner
from tfx.proto import example_gen_pb2
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
import os

## Set Variable

In [6]:
PIPELINE_NAME = "muhammad_yafi_079-pipeline"
SCHEMA_PIPELINE_NAME = "student-schema"

#Directory untuk menyimpan artifact yang akan dihasilkan
PIPELINE_ROOT = os.path.join('pipelines', PIPELINE_NAME)

# Path to a SQLite DB file to use as an MLMD storage.
METADATA_PATH = os.path.join('metadata', PIPELINE_NAME, 'metadata.db')

# Output directory where created models from the pipeline will be exported.
SERVING_MODEL_DIR = os.path.join('serving_model', PIPELINE_NAME)

# from absl import logging
# logging.set_verbosity(logging.INFO)

In [7]:
DATA_ROOT = "data"

In [8]:
import pandas as pd
df = pd.read_csv(r'C:\Users\ANANG\MLOps1\Yafi\data\Student.csv')
df

,StudentID,Age,Gender,Ethnicity,ParentalEducation,StudyTimeWeekly,Absences,Tutoring,ParentalSupport,Extracurricular,Sports,Music,Volunteering,GPA,GradeClass
0,1001,17,1,0,2,19.833723,7,1,2,0,0,1,0,2.929196,2.0
1,1002,18,0,0,1,15.408756,0,0,1,0,0,0,0,3.042915,1.0
2,1003,15,0,2,3,4.210570,26,0,2,0,0,0,0,0.112602,4.0
3,1004,17,1,0,3,10.028829,14,0,3,1,0,0,0,2.054218,3.0
4,1005,17,1,0,2,4.672495,17,1,3,0,0,0,0,1.288061,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2387,3388,18,1,0,3,10.680555,2,0,4,1,0,0,0,3.455509,0.0
2388,3389,17,0,0,1,7.583217,4,1,4,0,1,0,0,3.279150,4.0
2389,3390,16,1,0,2,6.805500,20,0,2,0,0,0,1,1.142333,2.0
2390,3391,16,1,1,0,12.416653,17,0,2,0,1,1,0,1.803297,1.0


In [9]:
interactive_context = InteractiveContext(pipeline_root=PIPELINE_ROOT)

Data Ingestion
ExampleGen
Code ini mengatur konfigurasi output untuk CsvExampleGen, yang membagi data dari DATA_ROOT menjadi dua bagian: train dengan 8 hash buckets dan eval dengan 2 hash buckets, untuk digunakan dalam konteks interaktif dalam pipeline deteksi stres.

In [10]:
output = example_gen_pb2.Output(
    split_config = example_gen_pb2.SplitConfig(splits=[
        example_gen_pb2.SplitConfig.Split(name="train", hash_buckets=8),
        example_gen_pb2.SplitConfig.Split(name="eval", hash_buckets=2)
    ])
)
example_gen = CsvExampleGen(input_base=DATA_ROOT, output_config=output)

In [11]:
interactive_context.run(example_gen)

c:\Users\ANANG\MLOps1\env\lib\site-packages\setuptools\distutils_patch.py:25: UserWarning: Distutils was imported before Setuptools. This usage is discouraged and may exhibit undesirable behaviors or errors. Please use Setuptools' objects directly or at least import Setuptools first.
  warnings.warn(


ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 1
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=CsvExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}))

StatisticGen
Code ini menjalankan StatisticsGen untuk menghasilkan statistik deskriptif dari data yang dihasilkan oleh example_gen dalam konteks interaktif pada pipeline deteksi stres.

In [12]:
statistics_gen = StatisticsGen(
    examples=example_gen.outputs["examples"]
)
 
 
interactive_context.run(statistics_gen)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 2
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}))

Code ini akan menampilkan output statistik yang dihasilkan oleh komponen StatisticsGen dari pipeline deteksi stres dalam konteks interaktif.

In [13]:
interactive_context.show(statistics_gen.outputs["statistics"])


SchemaGen
Code ini digunakan untuk menjalankan komponen SchemaGen dalam pipeline deteksi stres, yang menghasilkan skema data berdasarkan statistik yang dihasilkan sebelumnya oleh komponen StatisticsGen.

In [14]:
schema_gen = SchemaGen(    statistics=statistics_gen.outputs["statistics"]
)
interactive_context.run(schema_gen)

ExecutionResult(
    component_id: SchemaGen
    execution_id: 3
    outputs:
        schema: OutputChannel(artifact_type=Schema, producer_component_id=SchemaGen, output_key=schema, additional_properties={}, additional_custom_properties={}))

Code ini digunakan untuk menampilkan skema yang dihasilkan oleh komponen SchemaGen dalam pipeline deteksi stres.

In [15]:
interactive_context.show(schema_gen.outputs["schema"])


,Type,Presence,Valency,Domain
Feature name,,,,
'Absences',INT,required,,-
'Age',INT,required,,-
'Ethnicity',INT,required,,-
'Extracurricular',INT,required,,-
'GPA',FLOAT,required,,-
'Gender',INT,required,,-
'GradeClass',FLOAT,required,,-
'Music',INT,required,,-
'ParentalEducation',INT,required,,-


ExampleValidator
Code ini digunakan untuk menjalankan komponen ExampleValidator dalam pipeline deteksi stres, yang akan mengevaluasi contoh data berdasarkan statistik yang dihasilkan dan skema yang didefinisikan sebelumnya.

In [16]:
example_validator = ExampleValidator(
    statistics=statistics_gen.outputs['statistics'],
    schema=schema_gen.outputs['schema']
)
interactive_context.run(example_validator)

ExecutionResult(
    component_id: ExampleValidator
    execution_id: 4
    outputs:
        anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=ExampleValidator, output_key=anomalies, additional_properties={}, additional_custom_properties={}))

Code ini digunakan untuk menampilkan anomali yang terdeteksi oleh komponen ExampleValidator berdasarkan evaluasi terhadap statistik dan skema yang telah didefinisikan sebelumnya dalam pipeline deteksi stres.

In [17]:
interactive_context.show(example_validator.outputs['anomalies'])


Data Preprocessing
Transform
Code ini menginisialisasi dan menjalankan komponen Transform dalam pipeline deteksi stres menggunakan TFX. Komponen ini bertanggung jawab untuk mengambil contoh data dari example_gen, menerapkan transformasi yang didefinisikan dalam modul transformasi yang disediakan, dan menghasilkan output yang telah ditransformasi untuk digunakan dalam langkah-langkah selanjutnya dari pipeline.

In [18]:
TRANSFORM_MODULE_FILE = "Tutor_Detection_transform.py"

In [19]:
%%writefile {TRANSFORM_MODULE_FILE}

import tensorflow as tf

LABEL_KEY = "Tutoring"
FEATURE_KEY = "GPA"

def transformed_name(key):
    """Renaming transformed features"""
    return key + "_xf"

def preprocessing_fn(inputs):
    """
    Preprocess input features into transformed features

    Args:
        inputs: map from feature keys to raw features.

    Return:
        outputs: map from feature keys to transformed features.
    """

    outputs = {}

    # Directly use the FEATURE_KEY as a float without conversion to string
    outputs[transformed_name(FEATURE_KEY)] = tf.cast(inputs[FEATURE_KEY], tf.float32)

    outputs[transformed_name(LABEL_KEY)] = tf.cast(inputs[LABEL_KEY], tf.int64)

    return outputs


Writing Tutor_Detection_transform.py


In [20]:
transform = Transform(
    examples=example_gen.outputs['examples'],
    schema=schema_gen.outputs['schema'],
    module_file=os.path.abspath(TRANSFORM_MODULE_FILE)
)
interactive_context.run(transform)

Instructions for updating:
Use ref() instead.


Instructions for updating:
Use ref() instead.


INFO:tensorflow:Assets written to: pipelines\muhammad_yafi_079-pipeline\Transform\transform_graph\5\.temp_path\tftransform_tmp\091ea92b55374a36b513d0fad06b8ffb\assets


INFO:tensorflow:Assets written to: pipelines\muhammad_yafi_079-pipeline\Transform\transform_graph\5\.temp_path\tftransform_tmp\091ea92b55374a36b513d0fad06b8ffb\assets


INFO:tensorflow:struct2tensor is not available.


...

INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:tensorflow_text is not available.


ExecutionResult(
    component_id: Transform
    execution_id: 5
    outputs:
        transform_graph: OutputChannel(artifact_type=TransformGraph, producer_component_id=Transform, output_key=transform_graph, additional_properties={}, additional_custom_properties={})
        transformed_examples: OutputChannel(artifact_type=Examples, producer_component_id=Transform, output_key=transformed_examples, additional_properties={}, additional_custom_properties={})
        updated_analyzer_cache: OutputChannel(artifact_type=TransformCache, producer_component_id=Transform, output_key=updated_analyzer_cache, additional_properties={}, additional_custom_properties={})
        pre_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=pre_transform_schema, additional_properties={}, additional_custom_properties={})
        pre_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=pre_transform_stats, additional_properties={}, additional_custom_properties={})
        post_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=post_transform_schema, additional_properties={}, additional_custom_properties={})
        post_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=post_transform_stats, additional_properties={}, additional_custom_properties={})
        post_transform_anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=Transform, output_key=post_transform_anomalies, additional_properties={}, additional_custom_properties={}))

Tuner
Code ini menginisialisasi dan menjalankan komponen Tuner dalam pipeline deteksi stres menggunakan TFX. Komponen ini bertanggung jawab untuk melakukan penyetelan parameter model menggunakan Hyperband tuner dengan definisi model yang disesuaikan berdasarkan modul tuner yang diberikan. Hyperparameter tuning dilakukan berdasarkan metrik binary_accuracy dengan penggunaan data yang telah ditransformasi dari transform.outputs["transformed_examples"] dan skema yang dihasilkan dari schema_gen.outputs["schema"].

In [21]:
TUNER_MODULE_FILE = "Tutor_detection_tuner.py"


In [22]:
%%writefile {TUNER_MODULE_FILE}

from typing import Any, Dict, NamedTuple, Text
import keras_tuner as kt
import tensorflow as tf
import tensorflow_transform as tft
from keras_tuner.engine import base_tuner
from tensorflow.keras import layers
from tfx.components.trainer.fn_args_utils import FnArgs

LABEL_KEY = "Tutoring"
FEATURE_KEY = "GPA"
NUM_EPOCHS = 5

# Define parameter tuning with early stopping callbacks
TunerResult = NamedTuple(
    "TunerFnResult", [("tuner", base_tuner.BaseTuner), ("fit_kwargs", Dict[Text, Any])]
)
es = tf.keras.callbacks.EarlyStopping(
    monitor="val_binary_accuracy",
    mode="max",
    verbose=1,
    patience=10,
    restore_best_weights=True,
)


# Create utilities function
def transformed_name(key):
    """Renaming transformed features"""
    return key + "_xf"


def gzip_reader_fn(filenames):
    return tf.data.TFRecordDataset(filenames, compression_type="GZIP")


def input_fn(
    file_pattern, tf_transform_output, num_epochs, batch_size=64
) -> tf.data.Dataset:
    """Get post_transform feature & create batches of data"""

    # Get post_transform feature spec
    transform_feature_spec = tf_transform_output.transformed_feature_spec().copy()

    # create batches of data
    dataset = tf.data.experimental.make_batched_features_dataset(
        file_pattern=file_pattern,
        batch_size=batch_size,
        features=transform_feature_spec,
        reader=gzip_reader_fn,
        num_epochs=num_epochs,
        label_key=transformed_name(LABEL_KEY),
    )
    return dataset


def model_builder(hp):
    """
    Builds the model and sets up the hyperparameters to tune.
    """

    ## Define parameter used for tuning model
    n_layers = hp.Int("n_layers", min_value=1, max_value=5, step=1)
    fc_units = hp.Int("fc_units", min_value=16, max_value=64, step=16)
    lr = hp.Choice("lr", values=[1e-2, 1e-3, 1e-4])

    inputs = tf.keras.Input(
        shape=(1,), name=transformed_name(FEATURE_KEY), dtype=tf.float32
    )

    x = inputs
    for _ in range(n_layers):
        x = layers.Dense(fc_units, activation="relu")(x)
        x = layers.Dropout(0.2)(x)

    outputs = layers.Dense(1, activation="sigmoid")(x)

    model = tf.keras.Model(inputs=inputs, outputs=outputs)

    model.compile(
        loss=tf.keras.losses.BinaryCrossentropy(),
        optimizer=tf.keras.optimizers.Adam(lr),
        metrics=[tf.keras.metrics.BinaryAccuracy()],
    )

    # print(model)
    model.summary()
    return model


def tuner_fn(fn_args: FnArgs):
    # Ensure transform_graph_path is correct
    tf_output = tft.TFTransformOutput(fn_args.transform_graph_path)

    # Ensure train_files and eval_files paths are correct
    train_set = input_fn(fn_args.train_files[0], tf_output, NUM_EPOCHS)
    eval_set = input_fn(fn_args.eval_files[0], tf_output, NUM_EPOCHS)

    # Define tuner search strategy
    tuner = kt.Hyperband(
        hypermodel=model_builder,
        objective=kt.Objective("binary_accuracy", direction="max"),
        max_epochs=NUM_EPOCHS,
        factor=3,
        directory=fn_args.working_dir,
        project_name="kt_hyperband",
    )

    return TunerResult(
        tuner=tuner,
        fit_kwargs={
            "callbacks": [es],
            "x": train_set,
            "validation_data": eval_set,
            "steps_per_epoch": fn_args.train_steps,
            "validation_steps": fn_args.eval_steps,
        },
    )


Writing Tutor_detection_tuner.py


In [23]:
from tfx.proto import example_gen_pb2, pusher_pb2, trainer_pb2

tuner = Tuner(
    module_file=TUNER_MODULE_FILE,
    examples=transform.outputs["transformed_examples"],
    transform_graph=transform.outputs["transform_graph"],
    schema=schema_gen.outputs["schema"],
    train_args=trainer_pb2.TrainArgs(splits=["train"], num_steps=20),
    eval_args=trainer_pb2.EvalArgs(splits=["eval"], num_steps=10),
)

interactive_context.run(tuner)

Trial 10 Complete [00h 00m 02s]
binary_accuracy: 0.707812488079071

Best binary_accuracy So Far: 0.7281249761581421
Total elapsed time: 00h 00m 17s
INFO:tensorflow:Oracle triggered exit


INFO:tensorflow:Oracle triggered exit


Results summary
Results in pipelines\muhammad_yafi_079-pipeline\.temp\6\kt_hyperband
Showing 10 best trials
Trial summary
Hyperparameters:
n_layers: 3
fc_units: 64
lr: 0.001
tuner/epochs: 2
tuner/initial_epoch: 0
tuner/bracket: 1
tuner/round: 0
Score: 0.7281249761581421
Trial summary
Hyperparameters:
n_layers: 4
fc_units: 48
lr: 0.001
tuner/epochs: 2
tuner/initial_epoch: 0
tuner/bracket: 1
tuner/round: 0
Score: 0.72265625
Trial summary
Hyperparameters:
n_layers: 3
fc_units: 64
lr: 0.001
tuner/epochs: 5
tuner/initial_epoch: 2
tuner/bracket: 1
tuner/round: 1
tuner/trial_id: 0004
Score: 0.719531238079071
Trial summary
Hyperparameters:
n_layers: 4
fc_units: 32
lr: 0.01
tuner/epochs: 5
tuner/initial_epoch: 0
tuner/bracket: 0
tuner/round: 0
Score: 0.71484375
Trial summary
Hyperparameters:
n_layers: 2
fc_units: 64
lr: 0.01
tuner/epochs: 5
tuner/initial_epoch: 0
tuner/bracket: 0
tuner/round: 0
Score: 0.7132812738418579
Trial summary
Hyperparameters:
n_layers: 4
fc_units: 48
lr: 0.001
tuner/epo

ExecutionResult(
    component_id: Tuner
    execution_id: 6
    outputs:
        best_hyperparameters: OutputChannel(artifact_type=HyperParameters, producer_component_id=Tuner, output_key=best_hyperparameters, additional_properties={}, additional_custom_properties={})
        tuner_results: OutputChannel(artifact_type=TunerResults, producer_component_id=Tuner, output_key=tuner_results, additional_properties={}, additional_custom_properties={}))

Trainer
Code ini berfungsi sebagai modul trainer untuk melatih model machine learning dalam pipeline deteksi stres menggunakan TensorFlow Extended (TFX). Modul ini membangun model berdasarkan parameter yang diatur, melatihnya dengan data yang telah ditransformasi, dan menyimpan model yang telah dilatih ke dalam direktori serving_model_dir. Kemudian, modul trainer dijalankan dalam konteks interaktif TFX untuk melatih dan mengevaluasi model menggunakan data yang telah dipisahkan ke dalam subset train dan eval berdasarkan konfigurasi yang diberikan.

In [24]:
TRAINER_MODULE_FILE = "Tutor_detection_trainer.py"

In [25]:
%%writefile {TRAINER_MODULE_FILE}

import os
import tensorflow as tf
import tensorflow_transform as tft
from tensorflow.keras import layers
from tfx.components.trainer.fn_args_utils import FnArgs

LABEL_KEY = "Tutoring"
FEATURE_KEY = "GPA"


def transformed_name(key):
    """Renaming transformed features"""
    return key + "_xf"


def gzip_reader_fn(filenames):
    """Loads compressed data"""
    return tf.data.TFRecordDataset(filenames, compression_type="GZIP")


def input_fn(
    file_pattern, tf_transform_output, num_epochs, batch_size=64
) -> tf.data.Dataset:
    """Get post_tranform feature & create batches of data"""

    # Get post_transform feature spec
    transform_feature_spec = tf_transform_output.transformed_feature_spec().copy()

    # create batches of data
    dataset = tf.data.experimental.make_batched_features_dataset(
        file_pattern=file_pattern,
        batch_size=batch_size,
        features=transform_feature_spec,
        reader=gzip_reader_fn,
        num_epochs=num_epochs,
        label_key=transformed_name(LABEL_KEY),
    )
    return dataset


def model_builder(hp):
    """Build machine learning model"""
    inputs = tf.keras.Input(
        shape=(1,), name=transformed_name(FEATURE_KEY), dtype=tf.float32
    )

    x = inputs
    for _ in range(hp.get("n_layers")):
        x = layers.Dense(hp.get("fc_units"), activation="relu")(x)
        x = layers.Dropout(0.2)(x)

    outputs = layers.Dense(1, activation="sigmoid")(x)

    model = tf.keras.Model(inputs=inputs, outputs=outputs)

    model.compile(
        loss="binary_crossentropy",
        optimizer=tf.keras.optimizers.Adam(hp.get("lr")),
        metrics=[tf.keras.metrics.BinaryAccuracy()],
    )

    model.summary()
    return model


def _get_serve_tf_examples_fn(model, tf_transform_output):
    model.tft_layer = tf_transform_output.transform_features_layer()

    @tf.function
    def serve_tf_examples_fn(serialized_tf_examples):
        feature_spec = tf_transform_output.raw_feature_spec()

        feature_spec.pop(LABEL_KEY)

        parsed_features = tf.io.parse_example(serialized_tf_examples, feature_spec)

        transformed_features = model.tft_layer(parsed_features)

        # get predictions using the transformed features
        return model(transformed_features)

    return serve_tf_examples_fn


def run_fn(fn_args: FnArgs) -> None:
    log_dir = os.path.join(os.path.dirname(fn_args.serving_model_dir), "logs")

    tensorboard_callback = tf.keras.callbacks.TensorBoard(
        log_dir=log_dir, update_freq="batch"
    )

    es = tf.keras.callbacks.EarlyStopping(
        monitor="val_binary_accuracy",
        mode="max",
        verbose=1,
        patience=10,
        restore_best_weights=True,
    )
    mc = tf.keras.callbacks.ModelCheckpoint(
        fn_args.serving_model_dir,
        monitor="val_binary_accuracy",
        mode="max",
        verbose=1,
        save_best_only=True,
    )

    # Debugging: Log the contents of fn_args
    print("fn_args:", fn_args)
    
    # Check if hyperparameters are present
    if fn_args.hyperparameters:
        hp = fn_args.hyperparameters.get("values")
    else:
        # Set default hyperparameters if not present
        hp = {
            "embedding_dim": 64,  # Not used in this version
            "lstm_units": 64,  # Not used in this version
            "n_layers": 2,
            "fc_units": 32,
            "lr": 0.001,
            "tuner/epochs": 5,
        }

    # Load the transform output
    tf_transform_output = tft.TFTransformOutput(fn_args.transform_graph_path)

    # Create batches of data
    train_set = input_fn(
        fn_args.train_files, tf_transform_output, hp.get("tuner/epochs")
    )
    val_set = input_fn(fn_args.eval_files, tf_transform_output, hp.get("tuner/epochs"))

    # Build the model
    model = model_builder(hp)

    # Train the model
    model.fit(
        x=train_set,
        validation_data=val_set,
        callbacks=[tensorboard_callback, es, mc],
        steps_per_epoch=fn_args.train_steps,
        validation_steps=fn_args.eval_steps,
        epochs=hp.get("tuner/epochs"),
    )
    signatures = {
        "serving_default": _get_serve_tf_examples_fn(
            model, tf_transform_output
        ).get_concrete_function(
            tf.TensorSpec(shape=[None], dtype=tf.string, name="examples")
        )
    }
    
    model.save(fn_args.serving_model_dir, save_format="tf", signatures=signatures)


Writing Tutor_detection_trainer.py


In [26]:
from tfx.proto import trainer_pb2

trainer = Trainer(
    module_file=os.path.abspath(TRAINER_MODULE_FILE),
    examples=transform.outputs['transformed_examples'],
    transform_graph=transform.outputs['transform_graph'],
    schema=schema_gen.outputs['schema'],
    train_args=trainer_pb2.TrainArgs(splits=['train']),
    eval_args=trainer_pb2.EvalArgs(splits=['eval'])
)

interactive_context.run(trainer)

fn_args: FnArgs(working_dir=None, train_files=['pipelines\\muhammad_yafi_079-pipeline\\Transform\\transformed_examples\\5\\Split-train\\*'], eval_files=['pipelines\\muhammad_yafi_079-pipeline\\Transform\\transformed_examples\\5\\Split-eval\\*'], train_steps=None, eval_steps=None, schema_path='pipelines\\muhammad_yafi_079-pipeline\\SchemaGen\\schema\\3\\schema.pbtxt', schema_file='pipelines\\muhammad_yafi_079-pipeline\\SchemaGen\\schema\\3\\schema.pbtxt', transform_graph_path='pipelines\\muhammad_yafi_079-pipeline\\Transform\\transform_graph\\5', transform_output='pipelines\\muhammad_yafi_079-pipeline\\Transform\\transform_graph\\5', data_accessor=DataAccessor(tf_dataset_factory=<function get_tf_dataset_factory_from_artifact.<locals>.dataset_factory at 0x0000016D8D711CA0>, record_batch_factory=<function get_record_batch_factory_from_artifact.<locals>.record_batch_factory at 0x0000016D8D711C10>, data_view_decode_fn=None), serving_model_dir='pipelines\\muhammad_yafi_079-pipeline\\Trainer\

INFO:tensorflow:Assets written to: pipelines\muhammad_yafi_079-pipeline\Trainer\model\7\Format-Serving\assets


INFO:tensorflow:Assets written to: pipelines\muhammad_yafi_079-pipeline\Trainer\model\7\Format-Serving\assets


148/148 [==============================] - 3s 13ms/step - loss: 0.6394 - binary_accuracy: 0.6776 - val_loss: 0.6474 - val_binary_accuracy: 0.6680
Epoch 2/5
146/148 [============================>.] - ETA: 0s - loss: 0.6143 - binary_accuracy: 0.7062
Epoch 2: val_binary_accuracy did not improve from 0.66798
148/148 [==============================] - 1s 3ms/step - loss: 0.6139 - binary_accuracy: 0.7068 - val_loss: 0.6287 - val_binary_accuracy: 0.6680
Epoch 3/5
124/148 [========================>.....] - ETA: 0s - loss: 0.6042 - binary_accuracy: 0.7087
Epoch 3: val_binary_accuracy did not improve from 0.66798
148/148 [==============================] - 0s 3ms/step - loss: 0.6057 - binary_accuracy: 0.7067 - val_loss: 0.6236 - val_binary_accuracy: 0.6680
Epoch 4/5
126/148 [========================>.....] - ETA: 0s - loss: 0.6038 - binary_accuracy: 0.7059
Epoch 4: val_binary_accuracy did not improve from 0.66798
148/148 [==============================] - 0s 3ms/step - loss: 0.6027 - binary_accur

INFO:tensorflow:struct2tensor is not available.


...

INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:Assets written to: pipelines\muhammad_yafi_079-pipeline\Trainer\model\7\Format-Serving\assets


INFO:tensorflow:Assets written to: pipelines\muhammad_yafi_079-pipeline\Trainer\model\7\Format-Serving\assets


ExecutionResult(
    component_id: Trainer
    execution_id: 7
    outputs:
        model: OutputChannel(artifact_type=Model, producer_component_id=Trainer, output_key=model, additional_properties={}, additional_custom_properties={})
        model_run: OutputChannel(artifact_type=ModelRun, producer_component_id=Trainer, output_key=model_run, additional_properties={}, additional_custom_properties={}))

Resolver
Code ini menentukan resolver untuk memilih blessed model terbaru dalam konteks TFX menggunakan strategi LatestBlessedModelStrategy, sehingga dapat digunakan dalam tahap selanjutnya dalam pipeline.

In [27]:
from tfx.dsl.components.common.resolver import Resolver 
from tfx.dsl.input_resolution.strategies.latest_blessed_model_strategy import LatestBlessedModelStrategy 
from tfx.types import Channel 
from tfx.types.standard_artifacts import Model, ModelBlessing 
 
model_resolver = Resolver(
    strategy_class= LatestBlessedModelStrategy,
    model = Channel(type=Model),
    model_blessing = Channel(type=ModelBlessing)
).with_id('Latest_blessed_model_resolver')
 
interactive_context.run(model_resolver)

ExecutionResult(
    component_id: Latest_blessed_model_resolver
    execution_id: 8
    outputs:
        model: OutputChannel(artifact_type=Model, producer_component_id=Latest_blessed_model_resolver, output_key=model, additional_properties={}, additional_custom_properties={})
        model_blessing: OutputChannel(artifact_type=ModelBlessing, producer_component_id=Latest_blessed_model_resolver, output_key=model_blessing, additional_properties={}, additional_custom_properties={}))

Evaluator
Code ini mendefinisikan konfigurasi evaluasi untuk TensorFlow Model Analysis (TFMA), termasuk spesifikasi model, spesifikasi slicing, dan metrik yang akan dievaluasi seperti jumlah contoh, AUC, dan metrik klasifikasi biner lainnya dengan ambang batas tertentu untuk akurasi biner.

In [28]:
import tensorflow_model_analysis as tfma 
 
eval_config = tfma.EvalConfig(
    model_specs=[tfma.ModelSpec(label_key="Tutoring")],
    slicing_specs=[tfma.SlicingSpec()],
    metrics_specs=[
        tfma.MetricsSpec(
            metrics=[
                tfma.MetricConfig(class_name="ExampleCount"),
                tfma.MetricConfig(class_name="AUC"),
                tfma.MetricConfig(class_name="FalsePositives"),
                tfma.MetricConfig(class_name="TruePositives"),
                tfma.MetricConfig(class_name="FalseNegatives"),
                tfma.MetricConfig(class_name="TrueNegatives"),
                tfma.MetricConfig(
                    class_name="BinaryAccuracy",
                    threshold=tfma.MetricThreshold(
                        value_threshold=tfma.GenericValueThreshold(
                            lower_bound={"value": 0.5}
                        ),
                        change_threshold=tfma.GenericChangeThreshold(
                            direction=tfma.MetricDirection.HIGHER_IS_BETTER,
                            absolute={"value": 0.0001},
                        ),
                    ),
                ),
            ]
        )
    ],
)

In [29]:
from tfx.components import Evaluator
evaluator = Evaluator(
    examples=example_gen.outputs['examples'],
    model=trainer.outputs['model'],
    baseline_model=model_resolver.outputs['model'],
    eval_config=eval_config)
 
interactive_context.run(evaluator)

Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


ExecutionResult(
    component_id: Evaluator
    execution_id: 9
    outputs:
        evaluation: OutputChannel(artifact_type=ModelEvaluation, producer_component_id=Evaluator, output_key=evaluation, additional_properties={}, additional_custom_properties={})
        blessing: OutputChannel(artifact_type=ModelBlessing, producer_component_id=Evaluator, output_key=blessing, additional_properties={}, additional_custom_properties={}))

In [30]:
# Visualize the evaluation results
eval_result = evaluator.outputs['evaluation'].get()[0].uri
tfma_result = tfma.load_eval_result(eval_result)
tfma.view.render_slicing_metrics(tfma_result)
tfma.addons.fairness.view.widget_view.render_fairness_indicator(
    tfma_result
)

FairnessIndicatorViewer(slicingMetrics=[{'sliceValue': 'Overall', 'slice': 'Overall', 'metrics': {'binary_accu…

Model Deployment
Pusher
Code ini melakukan push model yang telah dilatih dan divalidasi ke direktori yang disebutkan dalam push_destination. Setelah push berhasil, kode mengirimkan permintaan HTTP GET ke endpoint TensorFlow Serving untuk mendapatkan informasi model yang telah di-push.

In [34]:
from tfx.components import Pusher 
from tfx.proto import pusher_pb2 
 
pusher = Pusher(
model=trainer.outputs['model'],
model_blessing=evaluator.outputs['blessing'],
push_destination=pusher_pb2.PushDestination(
    filesystem=pusher_pb2.PushDestination.Filesystem(
        base_directory='serving_model_dir/tutor-detection-model'))
 
)
 
interactive_context.run(pusher)

ExecutionResult(
    component_id: Pusher
    execution_id: 11
    outputs:
        pushed_model: OutputChannel(artifact_type=PushedModel, producer_component_id=Pusher, output_key=pushed_model, additional_properties={}, additional_custom_properties={}))

In [35]:
import requests
from pprint import PrettyPrinter
 
pp = PrettyPrinter()
pp.pprint(requests.get("http://localhost:8080/v1/models/tutor-detection-model").json())

{'model_version_status': [{'state': 'AVAILABLE',
                           'status': {'error_code': 'OK', 'error_message': ''},
                           'version': '1720704812'}]}
